In [0]:
df= spark.table("retail_bronze.superstore_bronze.gmart_bronze")
display(df.count())

In [0]:
from pyspark.sql.functions import *
df=df.withColumn("Quantity", (col("Quantity").try_cast("integer")))\
                        .where(col("Quantity") >= 1)\
                            .na.drop(subset=["City","Postal Code"])

In [0]:
display(df.groupBy("Order ID","Order Date","Ship Date","Ship Mode","Product ID").agg(count("*"))\
    .withColumn("Order Date",col("Order Date").cast("Timestamp"))\
        .withColumn("Ship Date",col("Ship Date").cast("Timestamp"))
            .select(col("Order ID").alias("Order_ID"),col("Order Date").alias("Order_Date"),\
                col("Ship Date").alias("Ship_Date"),col("Ship Mode").alias("Ship_Mode"),\
                    col("Product ID").alias("Product_ID"))\
                        .write.mode("append").saveAsTable("retail_silver.superstore_silver.date_silver")
        )


drop table retail_silver.superstore_silver.customers_silver;
drop table retail_silver.superstore_silver.date_silver;
drop table retail_silver.superstore_silver.product_silver;
drop table retail_bronze.superstore_bronze.gmart_bronze;

In [0]:
%sql
alter table retail_silver.superstore_silver.date_silver 
add constraint primary key references ("Order_ID","Product_ID")